# Project Luther

The goal of project luther is to create a linear regression model for supervised machine learning. We also need to incorporate web scrapping into the project to get practice finding what we need.  

I will be pulling data from Ted.com.  Specifically, I will be grabbing view count, tags and the published date.  We also will initiate our proxies here, and I will tell why we need them when they are employed below.

Packages needed for this portion:
 * Pandas
 * Numpy
 * BeautifulSoup
 * Requests
 * Regex

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import time
import csv
import pickle

In [2]:
#Proxies need to be changed each run until data is collected! All are from https://free-proxy-list.net/
proxies = {
    "http": 'http://54.153.105.149:80', 
    "https": 'http://54.153.105.149:80'
}
prox2 = {
    "http": 'http://108.61.23.81:8080', 
    "https": 'http://108.61.23.81:8080'
}
prox3 = {
    "http": 'http://52.179.123.54:3128', 
    "https": 'http://52.179.123.54:3128'
}
prox4 = {
    "http": 'http://35.169.210.105:3389', 
    "https": 'http://35.169.210.105:3389'
}
base = 'https://www.ted.com'
start_url = 'https://www.ted.com/talks'

response = rq.get(start_url)

In [38]:
page = response.text
mainpage = soup(page,"lxml")

In [39]:
talks = re.compile('/talks/')
views = re.compile("Related Tags")

In [3]:
def GetVid(mainlist):
    vid_str = []
    l = len(mainlist)-1
    for i in range(0,l,2):
        new_str = str(mainlist[i])
        vid_str.append(new_str.split('"')[5])
    return vid_str

In [41]:
#Getting all cases where we have a /talks url
list_1 = mainpage.find_all(href=talks)


In [42]:
#Pulling a list of all video end url links
lnk_list = GetVid(list_1)
lnk_list

['/talks/tracee_ellis_ross_a_woman_s_fury_holds_lifetimes_of_wisdom',
 '/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers',
 '/talks/clemantine_wamariya_war_and_what_comes_after',
 '/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes',
 '/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all',
 '/talks/zachary_r_wood_why_it_s_worth_listening_to_people_we_disagree_with',
 '/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico',
 '/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment',
 '/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell',
 '/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees',
 '/talks/robin_steinberg_what_if_we_ended_the_injustice_of_bail',
 '/talks/jaron_lanier_how_we_need_to_remake_the_internet',
 '/talks/malika_whitley_how_the_arts_help_homeless_youth_heal_and_build',
 '/talks/lera_boroditsky_how_language_shapes_the_way_we_think',
 '/talks/jose_andre

## Creating a list for all ted talks

The ted talks pagination has the url /talks?page=3, therefore we can simplify this to a 'talks?page=' + a list of numbers from 2 to 77 in '77'.  We need to cycle through group of proxies to avoid getting blocked from the site.  We never get permanently 'blocked', but we can get blocked temporarily.

In [43]:
page_lst = [str(i) for i in range(2,78)]
mid = '/talks?page='

In [44]:
#creating a list of all the search pages to pull video links from
url_list = [base+mid+i for i in page_lst]
url_list.insert(0,'https://www.ted.com/talks')

In [45]:
#Function I will call below for each video page
def GetVidLst(vurl, prox = None):
    resp = rq.get(vurl, prox)
    ppage = resp.text
    vidpage = soup(ppage,"lxml")
    vlist = vidpage.find_all(href=talks)
    vid_lnk_list = GetVid(vlist)
    return vid_lnk_list

In [46]:
#Creating a list of all the ted videos
fin = []
count=0
proxy = None
for i in url_list:
    count+=1
    fin +=GetVidLst(i, proxy)
    time.sleep(5)
    if count == 30 or count == 60 or count == 90 or count == 120:
        time.sleep(10)
        if count ==30:proxy = proxies
        if count ==60: proxy = prox2
        if count ==90: proxy = prox3
        if count ==120: 
            proxy = None 
            count = 10

## Create Ted Talk Dictionary

Now that we have our list of all the videos, we are going to create a dictionary with the following format:
{'Title':\['Views', \['Tags'\], Date Posted\]}.  

To accomplish this, I first did all the work on a single page, then I will create functions for each step and call them in a for loop over the list 'fin'.

In [32]:
#Changes views string to an integer
def view_to_int(viewstring):
    viewstring= viewstring.replace('\n', '').replace(',', '')
    return int(viewstring)

In [33]:
#Gets the view count from the site
def get_Views(site_text):
    views_list = []
    viewint=0
    views = site_text.find_all(class_=" d:n f-w:700 f:.9 f:1@xxl c:white ")
    str_list2 = str(views).split('<span')[1].split('\t')
    viewint = view_to_int(str_list2[5])
    return viewint

In [34]:
#Gets ratings from the HTML
def get_ratings(vidpage):
    super_comment = webpage.find_all(text = re.compile('\"ratings\":'))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split('\"ratings\":')
    super_comment = super_comment[1].split("\"name\":")
    target_ratings = ['Confusing', 'Funny',  'Unconvincing', 'Longwinded', 'Inspiring', 'Fascinating', 'Jaw-dropping']
    count = 0
    rate_dict = {}
    for i in super_comment:
        if len(i)>10:
            str_list = i.split(",")
            rate_name = str_list[0].replace("\"",'')
            rate_count = str_list[1].replace('\"count\":','').split('}')[0]
            if rate_name in target_ratings:
                rate_dict[rate_name] = float(rate_count)
    return rate_dict

In [35]:
#Cleaning up site text to get tags
def Cleanup(site_text):
    super_comment = site_text.find_all(text = re.compile('tag'))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split('tags')
    super_comment = super_comment[1].split("\"")
    tag=[]
    tag =[super_comment[i] for i in range(2,9,2)]
    return tag

In [36]:
#Gets published date form HTML
def getDate(site_text):
    super_comment = str(site_text.find_all(text = re.compile('recorded_at')))
    inter = re.findall(re.compile('"recorded_at"'), super_comment)
    intermed = super_comment.split(inter[0])
    intermed = intermed[1].split(':')
    date = intermed[1].replace('T00', '').replace('"', "")
    return date

In [85]:
#Creating a list of all the ted vides
data = {}
count=0
tot_views=[]
proxy = prox4
for i in fin:
    time.sleep(15)
    count+=1
    curl = base+i
    response = rq.get(curl, proxy)
    web_page = response.text
    webpage = soup(web_page,"lxml")
    date = getDate(webpage)
    tag_list = Cleanup(webpage)
    tot_views = get_Views(webpage)
    ratings = get_ratings(webpage)
    data[i] = [date,tot_views,tag_list,ratings]
    if count == 20 or count == 40 or count == 60 or count == 80:
        time.sleep(10)
        if count ==20:proxy = proxies
        if count ==40: proxy = prox2
        if count ==60: proxy = prox4
        if count ==80: 
            proxy = prox3
            count = 0



ValueError: could not convert string to float: '"viewed_count":772886'

In [87]:
#Writing a pickle file to save data

with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)

## To Pandas

Now that we have our dictionary of values, we are changing it to a pandas dataframe for easier manipulation.

There will then be some prelimnary exploring to see if it is formatted correctly or if more manipulation is needed.

In [41]:
#reading a pickle file reading to pick up where i left off in case something fails or i start over
with open("my_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)

## More Data

There is data from a kaggle that incorporates similar things above, and I will use it to enrich my data.  I will also substitute using it now because I am still troubleshooting pulling data from ted.com

In [9]:
#reading a pickle file from other notebook to get all the url lists I need to update data
with open("url_list.pkl", 'rb') as picklef: 
    kag_url = pickle.load(picklef)

In [37]:
#Creating a dictionary of all the info I need from the videos in the kag_url list
data = {}
count=0
tot_views=[]
proxy = prox2
for i in kag_url:
    time.sleep(30)
    count+=1
    response = rq.get(i, proxy)
    web_page = response.text
    webpage = soup(web_page,"lxml")
    date = getDate(webpage)
    tag_list = Cleanup(webpage)
    tot_views = get_Views(webpage)
    ratings = get_ratings(webpage)
    data[i] = [date,tot_views,tag_list,ratings]
    if count == 20 or count == 40 or count == 60 or count == 80:
        time.sleep(10)
        if count ==20:proxy = proxies
        if count ==40: proxy = prox2
        if count ==60: proxy = prox4
        if count ==80: 
            proxy = prox3
            count = 0

In [38]:
#Saving for future use, as the above code runs for at least 2 hours
with open('test_data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)